# 语义搜索
1. 读取文档， 按照页来管理，Document（文档的页）,List[Dpocument]
2. 分割文档，按照文本段落来管理，Chunck（段落）,List[Chunck]
3. 向量化： 文本段向量映射，需要嵌入模型来辅助
4. 向量库：把多个文本段存储到向量库


In [ ]:
from langchain_community.document_loaders import PyPDFLoader
from pathlib import Path

from langchain_text_splitters import RecursiveCharacterTextSplitter

PDF_FILE_PATH = "../data/中华人民共和国刑法.pdf"

loader = PyPDFLoader(file_path=Path(PDF_FILE_PATH))
docs = loader.load()
# print(len(docs), docs[0].__dict__)


# 文本分割器
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, add_start_index=True, chunk_overlap=200
)

all_splits = text_splitter.split_documents(docs)
# print(len(all_splits), all_splits[0].__dict__)

# 向量化：嵌入模型辅助
from langchain_ollama.embeddings import OllamaEmbeddings

embedder = OllamaEmbeddings(model="nomic-embed-text")
# vector_0 = embedder.embed_query(text=all_splits[0].page_content)
# print(len(vector_0), vector_0)

# 向量数据库存储
from langchain_chroma import Chroma

vector_db = Chroma(
    collection_name="china_law",
    persist_directory="../data/chroma_db",
    embedding_function=embedder,
)

ids = vector_db.add_documents(documents=all_splits)
print(len(ids), type(ids))


153 <class 'list'>


In [14]:
# 相似度查询
from langchain_chroma import Chroma

vector_db = Chroma(
    collection_name="china_law",
    persist_directory="../data/chroma_db",
    embedding_function=embedder,
)
user_input = "刑法总共有多少法条？"
# rets = vector_db.similarity_search(query=user_input)
# print(len(rets), rets[0].__dict__)
# for ret in rets:
#     print(ret.page_content)

# 带分数的相似度查询
results = vector_db.similarity_search_with_score(query=user_input)
print(len(results), results[0])


4 (Document(id='04985f47-a4ec-4038-b418-b1a98a5c2355', metadata={'author': 'YF-INT6', 'source': '..\\data\\中华人民共和国刑法.pdf', 'start_index': 0, 'page_label': '6', 'creationdate': '2021-06-08T11:30:48+08:00', 'creator': 'Microsoft® Word 2019', 'producer': 'Microsoft® Word 2019', 'total_pages': 153, 'moddate': '2021-06-08T11:30:48+08:00', 'page': 5}, page_content='－6－ \n何人有超越法律的特权。 \n第五条 刑罚的轻重，应当与犯罪分子所犯罪行和承担的刑\n事责任相适应。 \n第六条 凡在中华人民共和国领域内犯罪的，除法律有特别\n规定的以外，都适用本法。 \n凡在中华人民共和国船舶或者航空器内犯罪的，也适用本\n法。 \n犯罪的行为或者结果有一项发生在中华人民共和国领域内\n的，就认为是在中华人民共和国领域内犯罪。 \n第七条 中华人民共和国公民在中华人民共和国领域外犯\n本法规定之罪的，适用本法，但是按本法规定的最高刑为三年以\n下有期徒刑的，可以不予追究。 \n中华人民共和国国家工作人员和军人在中华人民共和国领\n域外犯本法规定之罪的，适用本法。 \n第八条 外国人在中华人民共和国领域外对中华人民共和\n国国家或者公民犯罪，而按本法规定的最低刑为三年以上有期徒\n刑的，可以适用本法，但是按照犯罪地的法律不受处罚的除外。 \n第九条 对于中华人民共和国缔结或者参加的国际条约所\n规定的罪行，中华人民共和国在所承担条约义务的范围内行使刑\n事管辖权的，适用本法。 \n第十条 凡在中华人民共和国领域外犯罪，依照本法应当负\n刑事责任的，虽然经过外国审判，仍然可以依照本法追究，但是'), 0.6078146696090698)


In [15]:
# 使用向量查询
from langchain_ollama.embeddings import OllamaEmbeddings

embedder = OllamaEmbeddings(model="nomic-embed-text")
user_input = "刑法总共有多少法条？"
query_embedding = embedder.embed_query(user_input)
results = vector_db.similarity_search_by_vector(query_embedding)
print(len(results), results[0].__dict__)


4 {'id': '04985f47-a4ec-4038-b418-b1a98a5c2355', 'metadata': {'total_pages': 153, 'page_label': '6', 'moddate': '2021-06-08T11:30:48+08:00', 'creator': 'Microsoft® Word 2019', 'source': '..\\data\\中华人民共和国刑法.pdf', 'author': 'YF-INT6', 'page': 5, 'creationdate': '2021-06-08T11:30:48+08:00', 'start_index': 0, 'producer': 'Microsoft® Word 2019'}, 'page_content': '－6－ \n何人有超越法律的特权。 \n第五条 刑罚的轻重，应当与犯罪分子所犯罪行和承担的刑\n事责任相适应。 \n第六条 凡在中华人民共和国领域内犯罪的，除法律有特别\n规定的以外，都适用本法。 \n凡在中华人民共和国船舶或者航空器内犯罪的，也适用本\n法。 \n犯罪的行为或者结果有一项发生在中华人民共和国领域内\n的，就认为是在中华人民共和国领域内犯罪。 \n第七条 中华人民共和国公民在中华人民共和国领域外犯\n本法规定之罪的，适用本法，但是按本法规定的最高刑为三年以\n下有期徒刑的，可以不予追究。 \n中华人民共和国国家工作人员和军人在中华人民共和国领\n域外犯本法规定之罪的，适用本法。 \n第八条 外国人在中华人民共和国领域外对中华人民共和\n国国家或者公民犯罪，而按本法规定的最低刑为三年以上有期徒\n刑的，可以适用本法，但是按照犯罪地的法律不受处罚的除外。 \n第九条 对于中华人民共和国缔结或者参加的国际条约所\n规定的罪行，中华人民共和国在所承担条约义务的范围内行使刑\n事管辖权的，适用本法。 \n第十条 凡在中华人民共和国领域外犯罪，依照本法应当负\n刑事责任的，虽然经过外国审判，仍然可以依照本法追究，但是', 'type': 'Document'}


In [ ]:
# 语义搜索封装成chain

from langchain_core.runnables import chain


@chain
def retriever(query: str):
    return vector_db.similarity_search(query)


rets = retriever.invoke("如何使用langchain")
print(len(rets), rets[0])


4 page_content='－5－  
第八节 组织、强迫、引诱、容留、介绍卖淫罪 
第九节 制作、贩卖、传播淫秽物品罪 
第七章 危害国防利益罪 
第八章 贪污贿赂罪 
第九章 渎职罪 
第十章 军人违反职责罪 
附则 
 
第一编 总则 
 
第一章 刑法的任务、基本原则和适用范围 
 
第一条 为了惩罚犯罪，保护人民，根据宪法，结合我国同
犯罪作斗争的具体经验及实际情况，制定本法。 
第二条 中华人民共和国刑法的任务，是用刑罚同一切犯罪
行为作斗争，以保卫国家安全，保卫人民民主专政的政权和社会
主义制度，保护国有财产和劳动群众集体所有的财产，保护公民
私人所有的财产，保护公民的人身权利、民主权利和其他权利，
维护社会秩序、经济秩序，保障社会主义建设事业的顺利进行。 
第三条 法律明文规定为犯罪行为的，依照法律定罪处刑；
法律没有明文规定为犯罪行为的，不得定罪处刑。 
第四条 对任何人犯罪，在适用法律上一律平等。不允许任' metadata={'creator': 'Microsoft® Word 2019', 'source': '..\\data\\中华人民共和国刑法.pdf', 'creationdate': '2021-06-08T11:30:48+08:00', 'start_index': 0, 'author': 'YF-INT6', 'page': 4, 'page_label': '5', 'moddate': '2021-06-08T11:30:48+08:00', 'producer': 'Microsoft® Word 2019', 'total_pages': 153}
